In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#### created the dimension so the fact table is clean, readable, stable, and change-proof.

####Data Reading

In [0]:
df=spark.sql("select distinct  payment_type from databricks_catalog.silver.nyc_taxi_trips where payment_type is not null")
df.display()

#####creating reference table

In [0]:
%sql
create table if not exists databricks_catalog.gold.ref_payment_type 
(payment_type int, payment_type_desc String)

In [0]:
%sql
insert into databricks_catalog.gold.ref_payment_type values(0,'Unkown'),(1,'Credit card'),(2,'Cash'),(3,'No charge'),(4,'Dispute')

In [0]:
df_ref=spark.sql("select * from databricks_catalog.gold.ref_payment_type")
df_ref.display()

In [0]:
df=df.join(df_ref, on='payment_type', how='left')
df.display()

#####Create surrogate key

In [0]:
df=df.withColumn('payment_sk',col('payment_type')).dropDuplicates(['payment_sk'])

####Upsert logic

In [0]:
from delta.tables import DeltaTable

if spark.catalog.tableExists('databricks_catalog.gold.dim_payment_type_scd1'):
    dt=DeltaTable.forName(spark,'databricks_catalog.gold.dim_payment_type_scd1')
    
    (
        dt.alias('t')
        .merge(
            df.alias('s'),
            't.payment_sk = s.payment_sk'
        )
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )

else:
    df.write.format('delta').saveAsTable('databricks_catalog.gold.dim_payment_type_scd1')